# Capstone Project - Predicting Neighborhood to open a Restaurant in Toronto, Canada (Week 2)
### Applied Data Science Capstone by IBM/Coursera


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **Toronto, Canada**.

Location plays a major role in the success of any restaurant. Therefore, a preliminary market analysis will help in determining a favorable location for the new restaurant. Factors like its neighborhood, or surrounded by the same cuisine restaurant, or have easy access to transportation, will affect sales.

Since there are lots of restaurants in Toronto we will try to detect **locations that are not already crowded with Italian restaurants**.

This project uses data science find out popular eateries present in the neighborhood and then to predict which neighborhood will be the best to open a restaurant.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants (any kind) in the neighborhood
* number of and distance to Italian restaurants in the neighborhood, if any

Following data sources will be needed to extract/generate the required information:
* The dataset was present at et: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,we use beautifulsoup library to get html data and then algorithmicaly extract dataframe. 
* to get coordinates we use https://cocl.us/Geospatial_data
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**

### Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c anaconda beautifulsoup4 --yes
!conda install -c anaconda lxml --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

## Data Extraction and Cleaning

In [2]:
#send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
page = BeautifulSoup(data,'html.parser')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [3]:
# append the data into the respective lists
for row in page.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [4]:
# create a new DataFrame from the three lists
df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [5]:
# drop cells with value Not assigned
df_dropna = df[df.Borough != "Not assigned\n"].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [6]:
# group neighborhoods in the same borough
df_group = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))


In [7]:
# removing extra spaces to make both dataframes similar
df_group['Neighborhood'] = df_group['Neighborhood'].str.replace(' /', ',')
df_group['PostalCode'] = df_group['PostalCode'].str.replace('\n', '')
df_group['Borough'] = df_group['Borough'].str.replace('\n', '')
df_group.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
csv_path = 'https://cocl.us/Geospatial_data'
df_ll = pd.read_csv(csv_path)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_ll.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_ll.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_group.iloc[0,0] == df_ll.iloc[0,0]

True

In [11]:
#merging dataframes
toronto_df = df_group.merge(df_ll, on="PostalCode", how='left')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
df_toronto= toronto_df[toronto_df['Borough'].astype(str).str.contains('Toronto')]

In [13]:
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


**Our Dataset with all the major neighborhoods of Toronto.**

### Using Geocode to get coordinates of Toronto

In [14]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## All major neighborhoods of Toronto with markers on map.

We will find venues around these locations in future.

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Using Foursquare API to get restaurants near these locations.

Defining Credentials

In [32]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20200619' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


### Getting nearby venues to City/Neighborhood of Toronto

We will find out all the Venues present within **500 radius** of our neighborhood and put them in a dataframe.

In [17]:
LIMIT = 10
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#to get new dataframe containing venues in neighborhood
city_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [19]:
city_venues.shape

(349, 7)

In [20]:
city_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


# Methodology <a name="Methodology"></a>

In this project we will direct our efforts on detecting areas of Toronto that have low Italian restaurant density. We will limit our analysis to 500 radius around major locations.

In first step we have collected the required **data: neighborhood, boroughs, postalcodes and location of Toronto, Canada** from Wikipedia using Beautifulsoup library and used geospatial data for location coordinates. From this data we collected Main city locations around Toronto where restaurants will get open. We have shown these locations on map with markers using Folium library.

Second step in our analysis will be calculation and exploration of "**restaurants**' around these major neighborhood locations. This new data help us to get a clear view of type of restaurants in the locality. We have also identified venue location, venue category, name, latitudes and longitudes of restaurants using **Foursqaure API**. Then we visualize these restaurant locations on Map of Toronto with **Italian Restaurants** markers different from others.

In third and final step we will focus on finding out promising areas by creating **clusters of locations that has any kind of restaurant present**. We will only take into consideration the locations with **no more than one restaurants in it's cluster**. We will present map of all clusters (using **k-means clustering**) of these restaurant locations with clear distinguish between currently present Italian restaurants from others and search for optimal venue location by finding best fit cluster.

# Analysis <a name="Analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's get all the **restaurants present on the neighborhood locations out of our vast Venue dataset:**

In [21]:
df_restaurants = city_venues[city_venues['Venue Category'].astype(str).str.contains('Restaurant')]
df_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
10,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
11,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
13,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant


In [22]:
df_restaurants.shape

(81, 7)

In [23]:
map_rest = folium.Map(location=[latitude, longitude], zoom_start=12)


# add markers to map
for lat, lng, label in zip(df_restaurants['Venue Latitude'], df_restaurants['Venue Longitude'], df_restaurants['Venue Category']):
    if (label == 'Italian Restaurant'):
        colour = 'yellow'
        fill_colour = '#f7f786'
    else :
        colour = 'blue'
        fill_colour='#3186cc'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color = colour,
        fill=True,
        fill_color = fill_colour,
        fill_opacity=0.7,
        parse_html=False).add_to(map_rest)      
    
map_rest

Here we can see markers of various restaurants located in the neighborhood of Toronto's main locations. 

**Yellow** markers show **Italian Restaurant** while **Blue** markers show others.
By this we can clearly visualize our final dataset which will be used to do analysis.

Now, we will use K-Means Clustering algorithm to form clusters of the locations to narrow down our search and find best fir clsuter with suitable locations.

In [24]:
from sklearn.cluster import KMeans

kclusters = 5

venue_xys = df_restaurants[['Venue Latitude', 'Venue Longitude']].values
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_xys)
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 3, 3], dtype=int32)

In [26]:
df_restaurants.insert(0, 'Cluster Labels', kmeans.labels_)

In [27]:
df_restaurants.head()

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,2,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
7,2,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
10,2,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
11,2,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
13,2,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant


## Let's visualize our Clusters on the map to determine Best locality for our Restaurant.

We will show Toronto Map divided into 5 clusters varying by colors. The Italian Restaurant location in the cluster will be highlighted by black outer ring. We will get a clear view of the Location which can be further considered as a winner.

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_restaurants['Venue Latitude'], df_restaurants['Venue Longitude'], df_restaurants['Venue Category'], df_restaurants['Cluster Labels']):
    #to distinguish between Italian and other restaurants we show them with black color outline.
    if (str(poi) == 'Italian Restaurant'):
        colour = 'black'
        fill_colour = rainbow[cluster-1]
    else :
        colour = rainbow[cluster-1]
        fill_colour= rainbow[cluster-1]
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colour,
        fill=True,
        fill_color=fill_colour,
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

As we can see, our map gets divided into 5 clusters each having restaurant locations marked with various colors. If we examine, we can conclude that **Top cluster(Cluster 3)** in green has only **1 Italian Restaurant**.

Let's further examine our suitable cluster...

From the map, we can see that Cluster 3 and Cluster 2 has less number of Italian Restaurants then other clusters. So, we will examine these two individualy to find a winner.

## Cluster 2

We can see that cluster 2 has '2' Italian Restaurants as row 10 and row 24. 

In [33]:
df_restaurants.loc[df_restaurants['Cluster Labels'] == 2, df_restaurants.columns[[1] + list(range(4, df_restaurants.shape[1]))]]

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
5,The Beaches,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
7,"The Danforth West, Riverdale",Pantheon,43.677621,-79.351434,Greek Restaurant
10,"The Danforth West, Riverdale",Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
11,"The Danforth West, Riverdale",Mezes,43.677962,-79.350196,Greek Restaurant
13,"The Danforth West, Riverdale",Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant
18,"India Bazaar, The Beaches West",The Burger's Priest,43.666731,-79.315556,Fast Food Restaurant
20,"India Bazaar, The Beaches West",O Sushi,43.666684,-79.316614,Sushi Restaurant
24,"India Bazaar, The Beaches West",Casa di Giorgio,43.666645,-79.315204,Italian Restaurant
344,"Business reply mail Processing Centre, South C...",Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant
346,"Business reply mail Processing Centre, South C...",The Green Wood,43.664728,-79.324117,Restaurant


## Cluster 3

At row 67, we can see that there is only one Italian Restaurant present. Therefore, this makes this cluster with least number of italian restaurants.

In [34]:
df_restaurants.loc[df_restaurants['Cluster Labels'] == 3, df_restaurants.columns[[1] + list(range(4, df_restaurants.shape[1]))]]

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
49,"North Toronto West, Lawrence Park",C'est Bon,43.716785,-79.400406,Chinese Restaurant
54,"North Toronto West, Lawrence Park",Sushi Shop,43.713861,-79.400093,Restaurant
56,"North Toronto West, Lawrence Park",Tio's Urban Mexican,43.714630,-79.400000,Mexican Restaurant
57,"North Toronto West, Lawrence Park",A&W,43.715149,-79.399944,Fast Food Restaurant
60,Davisville,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
61,Davisville,Zee Grill,43.704985,-79.388476,Seafood Restaurant
66,Davisville,Sakae Sushi,43.704944,-79.388704,Sushi Restaurant
67,Davisville,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant
72,"Summerhill West, Rathnelly, South Hill, Forest...",Daeco Sushi,43.687838,-79.395652,Sushi Restaurant
73,"Summerhill West, Rathnelly, South Hill, Forest...",Mary Be Kitchen,43.687708,-79.395062,Restaurant


### This concludes our analysis. We have Cluster 3 as a winner, with 12 neighborhoods for opening a new Italian restaurant. 

## Results and Discussion <a name="results"></a>

Our result shows that although there are many restaurants in Toronto, with some locations having more number of Italian Restaurants, there are still some localities with lesser competition for our stakeholders.

We came across two clusters which have less number of Italian restaurants than others. **Cluster 2 & 3**. These clusters have location of neighborhood and Venue name etc. By examining both clusters we find out that Cluster 2 has two Italian Restaurants and Cluster 3 has one. Therefore, we choose cluster 3 locations for our new restaurant as there was no other depending feature left.

By this result, North Toronto West, Lawrence Park are the most suitable localities as they do not have any Italian Restaurant there. If we wish to open a restaurant near Toronto central, Summerhill West will be the best choice.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Toronto areas with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. We have collected the required data: neighborhood, boroughs, postal codes and location of Toronto, Canada** from Wikipedia and used geospatial data for location coordinates.
Further, by retrieving venues from Foursquare data around major neighborhood locations,we visualize these restaurants on Map of Toronto. Then finding out promising areas by creating clusters of locations that has any kind of restaurant present (with clear distinguish between currently present Italian restaurants from others) in order to create major zones of interest (containing greatest number of potential locations) which can further to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.
